In [1]:
%AddJar file:/home/jovyan/aas/ch06-lsa/target/ch06-lsa-2.0.0-jar-with-dependencies.jar

Starting download from file:/home/jovyan/aas/ch06-lsa/target/ch06-lsa-2.0.0-jar-with-dependencies.jar
Finished download of ch06-lsa-2.0.0-jar-with-dependencies.jar


In [2]:
import edu.umd.cloud9.collection.XMLInputFormat
import edu.stanford.nlp.ling.CoreAnnotations.{LemmaAnnotation, SentencesAnnotation, TokensAnnotation}
import edu.stanford.nlp.pipeline.{Annotation, StanfordCoreNLP}
import edu.umd.cloud9.collection.wikipedia.WikipediaPage
import edu.umd.cloud9.collection.wikipedia.language.EnglishWikipediaPage
import java.util.Properties

import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.io.{LongWritable, Text}
import org.apache.spark.ml.feature.{CountVectorizer, IDF}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Dataset, SparkSession}

import scala.collection.JavaConverters._
import scala.collection.mutable.ArrayBuffer

val spark = SparkSession.builder().getOrCreate()
import spark.implicits._

spark = org.apache.spark.sql.SparkSession@32ae1ce3


i...


Spark local-1539613677795: Some(http://74107bd78b99:4040)

In [5]:
val path = "Data/Wikipedia-Geometry.xml"
@transient val conf = new Configuration()
conf.set(XMLInputFormat.START_TAG_KEY, "<page>")
conf.set(XMLInputFormat.END_TAG_KEY, "</page>")
val kvs = spark.sparkContext.newAPIHadoopFile(path, classOf[XMLInputFormat],
                                              classOf[LongWritable], classOf[Text], conf)
val rawXmls = kvs.map(_._2.toString).toDS()

path = Data/Wikipedia-Geometry.xml
conf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml
kvs = Data/Wikipedia-Geometry.xml NewHadoopRDD[4] at newAPIHadoopFile at <console>:66
rawXmls = [value: string]


[value: string]

In [6]:
rawXmls.show

+--------------------+
|               value|
+--------------------+
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
+--------------------+
only showing top 20 rows



In [7]:
def wikiXmlToPlainText(pageXml: String): Option[(String, String)] = {
// Wikipedia has updated their dumps slightly since Cloud9 was written,
// so this hacky replacement is sometimes required to get parsing to work.
val hackedPageXml = pageXml.replaceFirst(
"<text xml:space=\"preserve\" bytes=\"\\d+\">",
"<text xml:space=\"preserve\">")
val page = new EnglishWikipediaPage()
WikipediaPage.readPage(page, hackedPageXml)
if (page.isEmpty) None
else Some((page.getTitle, page.getContent))
}
val docTexts = rawXmls.filter(_ != null).flatMap(wikiXmlToPlainText)

docTexts = [_1: string, _2: string]


wikiXmlToPlainText: (pageXml: String)Option[(String, String)]


[_1: string, _2: string]

In [8]:
docTexts.show

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|Category:Discrete...|Category:Discrete...|
|Category:Incidenc...|Category:Incidenc...|
|Category:Metric g...|Category:Metric g...|
|Category:Integral...|Category:Integral...|
|Category:Conforma...|Category:Conforma...|
|Category:Trigonom...|Category:Trigonom...|
|Category:Convex g...|Category:Convex g...|
|             Coaxial|Coaxial
 
 

In g...|
|Category:Technica...|Category:Technica...|
|   Category:Symmetry|Category:Symmetry...|
|Category:Homogene...|Category:Homogene...|
|       Ambient space|Ambient space
An ...|
|Category:Duality ...|Category:Duality ...|
|          Superspace|Superspace
Supers...|
|     Geometry Center|Geometry Center
T...|
|          Dehn plane|Dehn plane
In geo...|
|Complex reflectio...|Complex reflectio...|
|Category:Geometri...|Category:Geometri...|
|    Lipschitz domain|Lipschitz domain
...|
|         Visual hull|Visual hul

In [9]:
def createNLPPipeline(): StanfordCoreNLP = {
    val props = new Properties()
    props.put("annotators","tokenize, ssplit, pos, lemma")
    new StanfordCoreNLP(props)
}

def isOnlyLetters(str: String): Boolean = {
    str.forall(c => Character.isLetter(c))
}

def plainTextToLemmas(text: String, stopWords: Set[String],
    pipeline: StanfordCoreNLP): Seq[String] = {
    val doc = new Annotation(text)
    pipeline.annotate(doc)
    
    val lemmas = new ArrayBuffer[String]()
    val sentences = doc.get(classOf[SentencesAnnotation])
    for (sentence <- sentences.asScala;
        token <- sentence.get(classOf[TokensAnnotation]).asScala){
        val lemma = token.get(classOf[LemmaAnnotation])
        if(lemma.length > 2 && !stopWords.contains(lemma)
          && isOnlyLetters(lemma)){
          lemmas += lemma.toLowerCase
        }
    }
    lemmas
}

createNLPPipeline: ()edu.stanford.nlp.pipeline.StanfordCoreNLP
isOnlyLetters: (str: String)Boolean
plainTextToLemmas: (text: String, stopWords: Set[String], pipeline: edu.stanford.nlp.pipeline.StanfordCoreNLP)Seq[String]


In [10]:
val stopWords = scala.io.Source.fromFile("stopwords.txt").getLines().toSet
val bStopWords = spark.sparkContext.broadcast(stopWords)

val terms: Dataset[(String, Seq[String])] = 
 docTexts.mapPartitions { iter =>
     val pipeline = createNLPPipeline()
     iter.map { case(title, contents) =>
        (title, plainTextToLemmas(contents, bStopWords.value, pipeline))
     }
 }

stopWords = Set(down, it's, ourselves, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, with, ...


Set(down, it's, ourselves, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, with, from, you've, they've, what's, wouldn't, both, could, its, under, which, you'd, an, be, here's, into, where, he'll, her, themselves, were, more, we'd, where's, they're, who's, between, aren't, ours, about, doesn't, how's, against, during, no, very, we, ha

In [11]:
val termsDF = terms.toDF("title","terms")

termsDF = [title: string, terms: array<string>]


[title: string, terms: array<string>]

In [12]:
val filtered = termsDF.where(size($"terms")>1)

filtered = [title: string, terms: array<string>]


[title: string, terms: array<string>]

In [13]:
filtered.show

Adding annotator tokenize
TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
Adding annotator ssplit
edu.stanford.nlp.pipeline.AnnotatorImplementations:
Adding annotator pos
Reading POS tagger model from edu/stanford/nlp/models/pos-tagger/english-left3words/english-left3words-distsim.tagger ... done [3.4 sec].
Adding annotator lemma
+--------------------+--------------------+
|               title|               terms|
+--------------------+--------------------+
|Category:Discrete...|[category, discre...|
|Category:Incidenc...|[category, incide...|
|Category:Metric g...|[category, metric...|
|Category:Integral...|[category, integr...|
|Category:Conforma...|[category, confor...|
|Category:Trigonom...|[category, trigon...|
|Category:Convex g...|[category, convex...|
|             Coaxial|[coaxial, geometr...|
|Category:Technica...|[category, techni...|
|   Category:Symmetry|[category, symmetry]|
|Category:Homogene...|[category, homoge...|
|       Ambient space|[a

In [15]:
val numTerms = 20000
val countVectorizer = new CountVectorizer().
    setInputCol("terms").
    setOutputCol("termFreqs").
    setVocabSize(numTerms)
val vocabModel = countVectorizer.fit(filtered)
val docTermFreqs = vocabModel.transform(filtered)

Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma


numTerms = 20000
countVectorizer = cntVec_024dfea34f89
vocabModel = cntVec_024dfea34f89
docTermFreqs = [title: string, terms: array<string> ... 1 more field]


[title: string, terms: array<string> ... 1 more field]

In [16]:
docTermFreqs.cache()

[title: string, terms: array<string> ... 1 more field]

In [18]:
val idf = new IDF().
    setInputCol("termFreqs").
    setOutputCol("tfidfVec")
val idfModel = idf.fit(docTermFreqs)
val docTermMatrix = idfModel.transform(docTermFreqs).select("title","tfidfVec")

idf = idf_e65f3f3b1f4d
idfModel = idf_e65f3f3b1f4d
docTermMatrix = [title: string, tfidfVec: vector]


lastException: Throwable = null


[title: string, tfidfVec: vector]

In [20]:
val termIds: Array[String] = vocabModel.vocabulary
val docIds = docTermFreqs.rdd.map(_.getString(0)).
    zipWithUniqueId().
    map(_.swap).
    collect().toMap

termIds = Array(point, geometry, space, line, plane, can, group, two, use, frac, one, mathcal, set, vector, also, surface, angle, shape, give, define, right, cdot, metric, function, coordinate, image, case, field, form, time, example, theory, method, parallel, mathbf, call, triangle, see, minkowski, axis, distance, problem, geodesic, circle, object, geometric, theorem, map, dimension, varphus, structure, number, area, base, partial, length, transformation, end, pattern, model, text, begin, first, follow, unit, phi, system, may, equation, term, measure, euclidean, computational, reference, order, curve, three, theta, figure, tangent, circ, leave, hyperbolic, study, definition, mathbb, volume, bmatrix, mathrm, include, symmetry, side, moir챕, take, anatomy, boundary, general...


[point, geometry, space, line, plane, can, group, two, use, frac, one, mathcal, set, vector, also, surface, angle, shape, give, define, right, cdot, metric, function, coordinate, image, case, field, form, time, example, theory, method, parallel, mathbf, call, triangle, see, minkowski, axis, distance, problem, geodesic, circle, object, geometric, theorem, map, dimension, varphus, structure, number, area, base, partial, length, transformation, end, pattern, model, text, begin, first, follow, unit, phi, system, may, equation, term, measure, euclidean, computational, reference, order, curve, three, theta, figure, tangent, circ, leave, hyperbolic, study, definition, mathbb, volume, bmatrix, mathrm, include, symmetry, side, moir챕, take, anatomy, boundary, general, show, different, manifold, flow, element, direction, since, construction, affine, layer, consider, describe, complex, every, contain, condition, left, vertical, sin, finite, real, analysis, let, lie, horizontal, draw, equal, doteq,

In [21]:
import org.apache.spark.mllib.linalg.{Vectors, Vector => MLLibVector}
import org.apache.spark.ml.linalg.{Vector => MLVector}

val vecRdd = docTermMatrix.select("tfidfVec").rdd.map { row =>
Vectors.fromML(row.getAs[MLVector]("tfidfVec"))
}

vecRdd = MapPartitionsRDD[83] at map at <console>:90


import org.apache.spark.mllib.linalg.{Vectors, Vector=>MLLibVector}
import org.apache.spark.ml.linalg.{Vector=>MLVector}


MapPartitionsRDD[83] at map at <console>:90

In [23]:
import org.apache.spark.mllib.linalg.distributed.RowMatrix

vecRdd.cache()
val mat = new RowMatrix(vecRdd)
val k = 500
val svd = mat.computeSVD(k, computeU=true)

mat = org.apache.spark.mllib.linalg.distributed.RowMatrix@58ec6685
k = 500
svd = 


SingularValueDecomposition(org.apache.spark.mllib.linalg.distributed.RowMatrix@1332039c,[845.2081084228365,609.1529753442101,564.6842643607005,559.2101423778738,548.6553995067528,511.23835814491963,412.766007478415,396.46555528730244,383.7185271255755,361.88530372314835,351.4536347936699,347.52809499198906,313.76997937231926,301.0098322022394,279.7090081178422,273.9884177657833,257.5025249407732,256.6486689028055,249.90770671283562,242.94415656687...


SingularValueDecomposition(org.apache.spark.mllib.linalg.distributed.RowMatrix@1332039c,[845.2081084228365,609.1529753442101,564.6842643607005,559.2101423778738,548.6553995067528,511.23835814491963,412.766007478415,396.46555528730244,383.7185271255755,361.88530372314835,351.4536347936699,347.52809499198906,313.76997937231926,301.0098322022394,279.7090081178422,273.9884177657833,257.5025249407732,256.6486689028055,249.90770671283562,242.94415656687787,241.8770035807372,238.1180788498697,227.9874061122662,209.13587030589542,205.13247881494334,200.41149720090746,196.58460381900608,193.7084225440146,192.45282980834148,189.3903549447565,182.35934820762256,176.25358929355565,172.57082138398408,171.86483435787997,170.24022865017707,168.31574943817614,160.42514979966646,155.68829790296473,152.02583484035569,149.70830942840445,145.97376154868638,143.0426251587742,141.10692714671222,137.46051101553994,135.519354354505,134.38431724477604,134.0179617373645,121.00365757209244,120.40521192082304,116

In [24]:
import org.apache.spark.mllib.linalg.{Matrix,
SingularValueDecomposition}
import org.apache.spark.mllib.linalg.distributed.RowMatrix

def topTermsInTopConcepts(
svd: SingularValueDecomposition[RowMatrix, Matrix],
numConcepts: Int,
numTerms: Int, termIds: Array[String])
: Seq[Seq[(String, Double)]] = {
val v = svd.V
val topTerms = new ArrayBuffer[Seq[(String, Double)]]()
val arr = v.toArray
for (i <- 0 until numConcepts) {
    val offs = i * v.numRows
    val termWeights = arr.slice(offs, offs + v.numRows).zipWithIndex
    val sorted = termWeights.sortBy(-_._1)
    topTerms += sorted.take(numTerms).map {
        case (score, id) => (termIds(id), score)
    }
}
topTerms
}

topTermsInTopConcepts: (svd: org.apache.spark.mllib.linalg.SingularValueDecomposition[org.apache.spark.mllib.linalg.distributed.RowMatrix,org.apache.spark.mllib.linalg.Matrix], numConcepts: Int, numTerms: Int, termIds: Array[String])Seq[Seq[(String, Double)]]


In [25]:
def topDocsInTopConcepts(
svd: SingularValueDecomposition[RowMatrix, Matrix],
numConcepts: Int, numDocs: Int, docIds:Map[Long, String])
: Seq[Seq[(String, Double)]]= {
val u = svd.U
val topDocs = new ArrayBuffer[Seq[(String, Double)]]()
for (i <- 0 until numConcepts) {
    val docWeights = u.rows.map(_.toArray(i)).zipWithUniqueId()
    topDocs += docWeights.top(numDocs).map{
        case (score, id) => (docIds(id), score)
    }
}
topDocs
}

topDocsInTopConcepts: (svd: org.apache.spark.mllib.linalg.SingularValueDecomposition[org.apache.spark.mllib.linalg.distributed.RowMatrix,org.apache.spark.mllib.linalg.Matrix], numConcepts: Int, numDocs: Int, docIds: Map[Long,String])Seq[Seq[(String, Double)]]


In [26]:
val topConceptTerms = topTermsInTopConcepts(svd, 6, 6, termIds)
val topConceptDocs = topDocsInTopConcepts(svd, 6, 6, docIds)
for ((terms, docs) <- topConceptTerms.zip(topConceptDocs)){
    println("Concept terms: " + terms.map(_._1).mkString(", "))
    println("Concept docs: " + docs.map(_._1).mkString(", "))
    println()
}

Concept terms: anatomy, geodesic, phi, mathcal, cdot, metric
Concept docs: Computational anatomy, Minkowski space, Busemann function, Large deformation diffeomorphic metric mapping, Riemannian metric and Lie bracket in computational anatomy, Bayesian model of computational anatomy

Concept terms: minkowski, spacetime, gamma, bmatrix, hadamard, busemann
Concept docs: Minkowski space, Busemann function, Minkowski diagram, Schema for horizontal dials, Snub (geometry), Line moir챕

Concept terms: bmatrix, snub, minkowski, spacetime, begin, end
Concept docs: Snub (geometry), Minkowski space, Minkowski diagram, Complex reflection group, Minkowski plane, Point-normal triangle

Concept terms: svg, dial, file, hour, method, line
Concept docs: Schema for horizontal dials, Line moir챕, Moir챕 pattern, Str채hle construction, Fat object, Vertical and horizontal

Concept terms: bmatrix, snub, geodesic, begin, end, coxeter
Concept docs: Snub (geometry), Busemann function, Schema for horizontal dials, Com

topConceptTerms = ArrayBuffer(ArraySeq((anatomy,0.2370458357671324), (geodesic,0.18946788763805247), (phi,0.18218295306454493), (mathcal,0.177265327610354), (cdot,0.17171520756112904), (metric,0.17017699711625997)), ArraySeq((minkowski,0.2637641563882057), (spacetime,0.17702276632479763), (gamma,0.16072211011332263), (bmatrix,0.1458341532272791), (hadamard,0.13719945753418727), (busemann,0.12752595423413346)), ArraySeq((bmatrix,0.4180199621287827), (snub,0.28107742163330224), (minkowski,0.19522822303902018), (spacetime,0.1330910386163647), (begin,0.1259199911021497), (end,0.12156075346744166)), ArraySeq((svg,0.4775750506383046), (dial,0.3253047446376867), (file,0.32116420133820234), (hour,0.3041183093033518), (method,0.1475527676710171), (line,0.1447118755043...


ArrayBuffer(ArraySeq((anatomy,0.2370458357671324), (geodesic,0.18946788763805247), (phi,0.18218295306454493), (mathcal,0.177265327610354), (cdot,0.17171520756112904), (metric,0.17017699711625997)), ArraySeq((minkowski,0.2637641563882057), (spacetime,0.17702276632479763), (gamma,0.16072211011332263), (bmatrix,0.1458341532272791), (hadamard,0.13719945753418727), (busemann,0.12752595423413346)), ArraySeq((bmatrix,0.4180199621287827), (snub,0.28107742163330224), (minkowski,0.19522822303902018), (spacetime,0.1330910386163647), (begin,0.1259199911021497), (end,0.12156075346744166)), ArraySeq((svg,0.4775750506383046), (dial,0.3253047446376867), (file,0.32116420133820234), (hour,0.3041183093033518), (method,0.1475527676710171), (line,0.14471187550436101)), ArraySeq((bmatrix,0.577315918838535), (snub,0.4134911083923771), (geodesic,0.16254455611126556), (begin,0.15254691608233586), (end,0.14988968096313698), (coxeter,0.1481086058702459)), ArraySeq((svg,0.1736657819683541), (dial,0.11829408336974